In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Forces synchronous CUDA errors
os.environ["TORCH_USE_CUDA_DSA"] = "1"    # Enables device-side assert tracking

In [3]:
!git clone https://github.com/microsoft/LLaVA-Med.git


Cloning into 'LLaVA-Med'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 451 (delta 53), reused 51 (delta 49), pack-reused 345 (from 2)
Receiving objects: 100% (451/451), 77.08 MiB | 29.34 MiB/s, done.
Resolving deltas: 100% (135/135), done.
Filtering content: 100% (10/10), 813.63 MiB | 176.55 MiB/s, done.


In [4]:
%cd LLaVA-Med

/content/LLaVA-Med


In [1]:
!pip install --upgrade pip setuptools wheel
!pip install -e .
!pip install numpy==1.26.4
!pip install torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.36.2 peft==0.4.0 accelerate==0.21.0


Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 154.1 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3

In [1]:
%cd LLaVA-Med
!mkdir -p checkpoints
!huggingface-cli download microsoft/llava-med-v1.5-mistral-7b --local-dir checkpoints/llava-med-v1.5-mistral-7b --local-dir-use-symlinks False


/content/LLaVA-Med
/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 12 files:   0% 0/12 [00:00<?, ?it/s]Downloading '.gitattributes' to 'checkpoints/llava-med-v1.5-mistral-7b/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 10.1MB/s]
Download complete. Moving file to checkpoints/llava-med-v1.5-mistral-7b/.gitattributes
Fetching 12 files:   8% 1/12 [00:00<00:01,  6.85it/s]
model-00002-of-00004.safetensors:   0% 0.00/5.00G [00:00<?, ?B/s]Downloading 'model-00003-of-00004.safetensors' to 'checkpoints/llava-med-v1.5-mistral-7b/.cache/huggingface/download/DaGOU-KRMVrY0aYktrsE34tL0Bs=.12b18ecdf8924d5fe28ada797fe6697fa60e62cba630759fbeb52975b261c4e2.incomplete'


config.json: 100% 1.41k/1.41k [00:00<00:00, 12.2MB/s]

# Load Base Model

In [1]:
%cd LLaVA-Med

/content/LLaVA-Med


In [2]:
from torch.utils.data import Dataset
from PIL import Image
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from llava.model.builder import load_pretrained_model
import pandas as pd
import os
import torch

# Step 1: Load pretrained LLaVA-Med model
tokenizer, base_model, image_processor, context_len = load_pretrained_model(
    model_path="checkpoints/llava-med-v1.5-mistral-7b",
    model_base=None,
    model_name="llava-med-v1.5-mistral-7b"
)

# Step 2: Ensure tokenizer has a valid pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.unk_token

# Step 3: Define LoRA config with BitFit (bias="all")
lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="all",  # Change if BitFit or Not
    task_type="CAUSAL_LM"
)

# Step 4: Apply LoRA + BitFit to base model
model = get_peft_model(base_model, lora_config)

# Step 5: Unfreeze LayerNorm parameters (optional but recommended)
for name, param in model.named_parameters():
    if any(norm in name.lower() for norm in ["norm", "ln"]):
        param.requires_grad = True

for name, param in model.named_parameters():
    if param.requires_grad and not name.endswith(".lora_A") and not name.endswith(".lora_B"):
        param.data = param.data.to(torch.float32)

# Step 6: Report trainable parameters
model.print_trainable_parameters()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at checkpoints/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.14.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.4.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.12.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.14.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.3.self_attn.k_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.22.mlp.fc1.weight', 'model.vision_tower.vision

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

KeyboardInterrupt: 

# Load PEFT Model

In [2]:
# @title Default title text
from torch.utils.data import Dataset
from PIL import Image
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path
from transformers import TrainingArguments, Trainer, AutoTokenizer
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel
from llava.model.builder import load_pretrained_model
import pandas as pd
import os
import torch

model_file_path = "/content/drive/MyDrive/lora_output/model (BitFit 2 Epoch)"
# Load PEFT configuration
peft_config = PeftConfig.from_pretrained(model_file_path)

# Infer model name
model_name = get_model_name_from_path(peft_config.base_model_name_or_path)

tokenizer, base_model, image_processor, context_len = load_pretrained_model(
    model_path=peft_config.base_model_name_or_path,
    model_base=None,
    model_name=model_name,
    load_8bit=False,
    load_4bit=False,
)

# Load the LoRA adapter weights
model = PeftModel.from_pretrained(base_model, model_file_path)

# Step 5: Unfreeze LayerNorm parameters (optional but recommended)
for name, param in model.named_parameters():
    if any(norm in name.lower() for norm in ["norm", "ln"]):
        param.requires_grad = True

for name, param in model.named_parameters():
    if param.requires_grad and not name.endswith(".lora_A") and not name.endswith(".lora_B"):
        param.data = param.data.to(torch.float32)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at checkpoints/llava-med-v1.5-mistral-7b were not used when initializing LlavaMistralForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.13.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.18.self_attn.k_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.self_attn.out_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.self_attn.k_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.16.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.mlp.fc2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.19.mlp.fc2.weight', 'model.

pytorch_model.bin:  47%|####7     | 1.54G/3.25G [00:00<?, ?B/s]

# Training

In [3]:
class PneumothoraxDataset(Dataset):
    def __init__(self, image_dir, csv_path, metadata_csv, tokenizer, image_processor):
        self.data = pd.read_csv(csv_path)
        self.metadata = pd.read_csv(metadata_csv).set_index("Image Index")[["Patient Age", "Patient Sex"]].to_dict(orient="index")
        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.image_processor = image_processor

        # Filter out missing images
        self.data = self.data[self.data.iloc[:, 0].apply(lambda f: os.path.exists(os.path.join(image_dir, f)))].reset_index(drop=True)
        print(f"Dataset initialized with {len(self.data)} valid samples")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = self.data.iloc[idx, 0]
        response_text = str(self.data.iloc[idx, 1]).strip().lower()
        label = int(self.data.iloc[idx, 1])

        meta = self.metadata.get(filename, {})
        age = meta.get('Patient Age', 'unknown')
        sex = meta.get('Patient Sex', 'unknown')

        # Create the conversation
        question = f"You are an expert radiologist. This chest X-ray is from a {age}-year-old {sex} patient. Does this chest X-ray show a pneumothorax? Answer 'Yes' or 'No' and provide an explanation."
        answer = "Yes, this X-ray shows signs of pneumothorax." if label == 1 else "No, there are no signs of pneumothorax in this X-ray."

        conv = conv_templates["mistral_instruct"].copy()
        conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
        conv.append_message(conv.roles[1], answer)
        conversation = conv.get_prompt()

        # Tokenize the conversation
        input_ids = tokenizer_image_token(conversation, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')

        # Create labels - mask the question part, only train on the answer
        labels = input_ids.clone()

        # Find where the assistant's response starts
        assistant_start_token = self.tokenizer.encode(conv.roles[1], add_special_tokens=False)[0]
        try:
            assistant_idx = (input_ids == assistant_start_token).nonzero(as_tuple=True)[0]
            if len(assistant_idx) > 0:
                # Mask everything before the assistant's response
                labels[:assistant_idx[0]] = -100
        except:
            # If we can't find the assistant token, mask first half as fallback
            labels[:len(labels)//2] = -100

        # Load and process image
        image_path = os.path.join(self.image_dir, filename)
        try:
            image = Image.open(image_path).convert("RGB")
            pixel_values = self.image_processor.preprocess(image, return_tensors="pt")["pixel_values"][0]
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
            # Create a blank image tensor
            pixel_values = torch.zeros((3, 224, 224))

        return {
            "input_ids": input_ids.flatten(),
            "labels": labels.flatten(),
            "pixel_values": pixel_values,
        }

In [4]:
train_dataset = PneumothoraxDataset(
    "/content/drive/MyDrive/preprocessed_pneumothorax/train",
    "/content/drive/MyDrive/preprocessed_pneumothorax/train.csv",
    "/content/drive/MyDrive/preprocessed_pneumothorax/pneumothorax_combined.csv",
    tokenizer, image_processor
)

val_dataset = PneumothoraxDataset(
    "/content/drive/MyDrive/preprocessed_pneumothorax/val",
    "/content/drive/MyDrive/preprocessed_pneumothorax/val.csv",
    "/content/drive/MyDrive/preprocessed_pneumothorax/pneumothorax_combined.csv",
    tokenizer, image_processor
)

def collate_fn(instances):
    input_ids = [instance['input_ids'] for instance in instances]
    labels = [instance['labels'] for instance in instances]
    images = [instance['pixel_values'] for instance in instances]

    # Pad sequences to the same length
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    labels = torch.nn.utils.rnn.pad_sequence(
        labels, batch_first=True, padding_value=-100
    )
    images = torch.stack(images, dim=0)

    return {
        'input_ids': input_ids,
        'labels': labels,
        'images': images,
        'attention_mask': input_ids.ne(tokenizer.pad_token_id),
    }

Dataset initialized with 2935 valid samples
Dataset initialized with 629 valid samples


In [5]:
from transformers import Trainer

class LlavaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        try:
            # Extract images (not pixel_values for LLaVA-Med)
            images = inputs.pop("images", None)

            # Forward pass with correct parameter name
            outputs = model(
                input_ids=inputs['input_ids'],
                attention_mask=inputs.get('attention_mask', None),
                labels=inputs['labels'],
                images=images
            )

            loss = outputs.loss
            return (loss, outputs) if return_outputs else loss

        except Exception as e:
            print(f"Error in compute_loss: {e}")
            print(f"Input shapes: input_ids={inputs['input_ids'].shape}, labels={inputs['labels'].shape}")
            if 'images' in inputs:
                print(f"images shape: {inputs['images'].shape}")
            raise e

In [6]:
import os
# Set environment variables for better debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

training_args = TrainingArguments(
    output_dir="./lora_output",
    per_device_train_batch_size=2,  # Increased batch size slightly
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,   # Added gradient accumulation
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    logging_strategy="steps",
    logging_steps=10,
    logging_dir="./logs",
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=50,
    fp16=True,
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    group_by_length=False,  # Disable to avoid issues with variable length sequences
)

trainer = LlavaTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn,
)

# Start training
trainer.train()



Step,Training Loss,Validation Loss
50,0.057900,0.058560
100,0.055500,0.058374
150,0.057000,0.058350
200,0.058200,0.058193
250,0.055400,0.058166
300,0.057500,0.058206
350,0.057300,0.058217


TrainOutput(global_step=367, training_loss=0.05701692641918276, metrics={'train_runtime': 2336.839, 'train_samples_per_second': 1.256, 'train_steps_per_second': 0.157, 'total_flos': 1.0951696728926208e+16, 'train_loss': 0.05701692641918276, 'epoch': 1.0})

In [7]:
model.save_pretrained("/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)")
tokenizer.save_pretrained("/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)")


('/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)/tokenizer_config.json',
 '/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)/special_tokens_map.json',
 '/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)/tokenizer.model',
 '/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)/added_tokens.json',
 '/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)/tokenizer.json')

# Inference

In [8]:
import os
import random
import pandas as pd
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from llava.mm_utils import tokenizer_image_token
from llava.conversation import conv_templates
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN

def generate_response(image_path, metadata=None):
    try:
        image = Image.open(image_path).convert('RGB')
        image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values']

        # Move to same device as model
        if hasattr(model, 'device'):
            image_tensor = image_tensor.to(model.device, dtype=torch.float32)
        else:
            image_tensor = image_tensor.to('cuda', dtype=torch.float32)

        # Create the same prompt format as training
        age = metadata.get('Patient Age', 'unknown') if metadata else 'unknown'
        sex = metadata.get('Patient Sex', 'unknown') if metadata else 'unknown'

        question = f"You are an expert radiologist. This chest X-ray is from a {age}-year-old {sex} patient. Does this chest X-ray show a pneumothorax? Answer 'Yes' or 'No' and provide an explanation."

        # Use the exact same conversation format as training
        conv = conv_templates["mistral_instruct"].copy()
        conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + question)
        conv.append_message(conv.roles[1], None)
        prompt_formatted = conv.get_prompt()

        # Tokenize using the same method as training
        input_ids = tokenizer_image_token(prompt_formatted, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt')

        # Ensure proper shape
        if input_ids.dim() == 1:
            input_ids = input_ids.unsqueeze(0)

        input_ids = input_ids.to(model.device)

        with torch.inference_mode():
            output_ids = model.base_model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=0.2,
                top_p=0.9,
                max_new_tokens=128,  # Reduced for more focused responses
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )

        # Decode only the new tokens
        input_token_len = input_ids.shape[1]

        # Handle case where output might be shorter than input
        if output_ids.shape[1] > input_token_len:
            outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
        else:
            outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
            # Remove the input part if it's still there
            if question in outputs:
                outputs = outputs.split(question)[-1]

        return outputs.strip()

    except Exception as e:
        print(f"Error in generate_response: {e}")
        return "Error generating response"

In [9]:
from llava.model.builder import load_pretrained_model
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/lora_output/model (BitFit 3 Epoch)")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlavaMistralForCausalLM(
      (model): LlavaMistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=1024, bias=False
   

In [10]:
import torch


test_folder = "/content/drive/MyDrive/preprocessed_pneumothorax/test"
csv_path = f"/content/drive/MyDrive/preprocessed_pneumothorax/test.csv"
metadata_csv_path = "/content/drive/MyDrive/preprocessed_pneumothorax/pneumothorax_combined.csv"

df = pd.read_csv(csv_path)
filename_to_label = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
metadata_df = pd.read_csv(metadata_csv_path)
metadata_lookup = metadata_df.set_index("Image Index")[["Patient Age", "Patient Sex"]].to_dict(orient="index")

true_labels, predicted_labels = [], []
shuffled_items = list(filename_to_label.items())
random.seed(42)
random.shuffle(shuffled_items)
model = model.float().eval()
for i, (filename, label) in enumerate(shuffled_items):
    image_path = os.path.join(test_folder, filename)
    if not os.path.exists(image_path):
        continue

    metadata = metadata_lookup.get(filename)
    try:
        response = generate_response(image_path, metadata)
        predicted = int(response.lower().startswith("yes"))
        true_labels.append(int(label))
        predicted_labels.append(predicted)

        print(f"Filename       : {filename}")
        print(f"Ground Truth   : {'Pneumothorax' if label == 1 else 'No Finding'}")
        print(f"Model Response : {response}")
        print("-" * 60)

        if (i + 1) % 10 == 0:
            tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels, labels=[0, 1]).ravel()
            total = len(true_labels)
            correct = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p])
            accuracy = correct / total
            precision = precision_score(true_labels, predicted_labels, zero_division=0)
            recall = recall_score(true_labels, predicted_labels, zero_division=0)
            specificity = tn / (tn + fp) if (tn + fp) != 0 else 0.0
            f1 = f1_score(true_labels, predicted_labels, zero_division=0)

            print(f"=== INTERMEDIATE RESULTS ({i + 1} samples) ===")
            print(f"Total samples processed: {total}")
            print(f"Correct predictions: {correct}")
            print(f"Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)\n")
            print(f"TP (Pneumothorax → Pneumothorax): {tp}")
            print(f"TN (No Finding → No Finding): {tn}")
            print(f"FP (No Finding → Pneumothorax): {fp}")
            print(f"FN (Pneumothorax → No Finding): {fn}\n")
            print(f"Precision: {precision:.4f} ({precision * 100:.2f}%)")
            print(f"Recall: {recall:.4f} ({recall * 100:.2f}%)")
            print(f"Specificity: {specificity:.4f} ({specificity * 100:.2f}%)")
            print(f"F1-Score: {f1:.4f} ({f1 * 100:.2f}%)")
            print("=" * 60)
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        continue

# === FINAL METRICS ===
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score,
    average_precision_score, balanced_accuracy_score, matthews_corrcoef
)
if true_labels and predicted_labels:
    tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels, labels=[0, 1]).ravel()
    total = len(true_labels)
    correct = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p])
    accuracy = correct / total
    precision = precision_score(true_labels, predicted_labels, zero_division=0)
    recall = recall_score(true_labels, predicted_labels, zero_division=0)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0.0
    f1 = f1_score(true_labels, predicted_labels, zero_division=0)

    # Additional metrics
    balanced_acc = balanced_accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)

    try:
        auc_roc = roc_auc_score(true_labels, predicted_labels)
    except ValueError:
        auc_roc = float('nan')  # Handle case with only one class

    try:
        auc_pr = average_precision_score(true_labels, predicted_labels)
    except ValueError:
        auc_pr = float('nan')

    print("\n=== FINAL ACCURACY RESULTS ===")
    print(f"Total samples processed: {total}")
    print(f"Correct predictions: {correct}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")

    print("\n=== CONFUSION MATRIX ===")
    print(f"TP (Pneumothorax → Pneumothorax): {tp}")
    print(f"TN (No Finding → No Finding): {tn}")
    print(f"FP (No Finding → Pneumothorax): {fp}")
    print(f"FN (Pneumothorax → No Finding): {fn}")

    print("\n=== METRICS ===")
    print(f"Precision: {precision:.4f} ({precision * 100:.2f}%)")
    print(f"Recall: {recall:.4f} ({recall * 100:.2f}%)")
    print(f"Specificity: {specificity:.4f} ({specificity * 100:.2f}%)")
    print(f"F1-Score: {f1:.4f} ({f1 * 100:.2f}%)")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print(f"AUC-PR: {auc_pr:.4f}")


Filename       : 00008304_001.png
Ground Truth   : Pneumothorax
Model Response : Yes, this X-ray shows signs of pneumothorax.
------------------------------------------------------------
Filename       : 00017972_046.png
Ground Truth   : Pneumothorax
Model Response : Yes, this X-ray shows signs of pneumothorax.
------------------------------------------------------------
Filename       : 00015892_003.png
Ground Truth   : Pneumothorax
Model Response : Yes, this X-ray shows signs of pneumothorax.
------------------------------------------------------------
Filename       : 00004581_006.png
Ground Truth   : Pneumothorax
Model Response : Yes, this X-ray shows signs of pneumothorax.
------------------------------------------------------------
Filename       : 00022651_016.png
Ground Truth   : Pneumothorax
Model Response : Yes, this X-ray shows signs of pneumothorax.
------------------------------------------------------------
Filename       : 00000484_000.png
Ground Truth   : No Finding
Mod

In [14]:
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score,
    average_precision_score, balanced_accuracy_score, matthews_corrcoef
)
if true_labels and predicted_labels:
    tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels, labels=[0, 1]).ravel()
    total = len(true_labels)
    correct = sum([1 for t, p in zip(true_labels, predicted_labels) if t == p])
    accuracy = correct / total
    precision = precision_score(true_labels, predicted_labels, zero_division=0)
    recall = recall_score(true_labels, predicted_labels, zero_division=0)
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0.0
    f1 = f1_score(true_labels, predicted_labels, zero_division=0)

    # Additional metrics
    balanced_acc = balanced_accuracy_score(true_labels, predicted_labels)
    mcc = matthews_corrcoef(true_labels, predicted_labels)

    try:
        auc_roc = roc_auc_score(true_labels, predicted_labels)
    except ValueError:
        auc_roc = float('nan')  # Handle case with only one class

    try:
        auc_pr = average_precision_score(true_labels, predicted_labels)
    except ValueError:
        auc_pr = float('nan')

    print("\n=== FINAL ACCURACY RESULTS ===")
    print(f"Total samples processed: {total}")
    print(f"Correct predictions: {correct}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")

    print("\n=== CONFUSION MATRIX ===")
    print(f"TP (Pneumothorax → Pneumothorax): {tp}")
    print(f"TN (No Finding → No Finding): {tn}")
    print(f"FP (No Finding → Pneumothorax): {fp}")
    print(f"FN (Pneumothorax → No Finding): {fn}")

    print("\n=== METRICS ===")
    print(f"Precision: {precision:.4f} ({precision * 100:.2f}%)")
    print(f"Recall: {recall:.4f} ({recall * 100:.2f}%)")
    print(f"Specificity: {specificity:.4f} ({specificity * 100:.2f}%)")
    print(f"F1-Score: {f1:.4f} ({f1 * 100:.2f}%)")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print(f"AUC-PR: {auc_pr:.4f}")



=== FINAL ACCURACY RESULTS ===
Total samples processed: 630
Correct predictions: 489
Accuracy: 0.7762 (77.62%)

=== CONFUSION MATRIX ===
TP (Pneumothorax → Pneumothorax): 254
TN (No Finding → No Finding): 235
FP (No Finding → Pneumothorax): 65
FN (Pneumothorax → No Finding): 76

=== METRICS ===
Precision: 0.7962 (79.62%)
Recall: 0.7697 (76.97%)
Specificity: 0.7833 (78.33%)
F1-Score: 0.7827 (78.27%)
Balanced Accuracy: 0.7765
MCC: 0.5524
AUC-ROC: 0.7765
AUC-PR: 0.7335
